In [1]:
import random
random.seed(0)

from transformers import AutoTokenizer, AutoModelForCausalLM
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TRANSFORMERS_CACHE'] = '/home/csgrad/sunilruf/'
path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/rank_64/rank_64_2/checkpoint-50"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(path, load_in_4bit=True, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
import json

json_file_path = '../../../ExTES.json'  # Change 'output.json' to your desired file name

# Write the list to the JSON file
with open(json_file_path, 'rb') as json_file:
    extes_data = json.load(json_file)

In [10]:
from datasets import load_dataset

dataset = load_dataset("thu-coai/esconv")

In [21]:
json.loads(dataset['test'][0]['text'])

{'experience_type': 'Current Experience',
 'emotion_type': 'depression',
 'problem_type': 'ongoing depression',
 'situation': 'I am always depressed and the upcoming holidays are making it a lot worse.',
 'survey_score': {'seeker': {'initial_emotion_intensity': '4',
   'empathy': '5',
   'relevance': '5',
   'final_emotion_intensity': '3'},
  'supporter': {'relevance': '5'}},
 'dialog': [{'text': 'Hello. How are you today?',
   'speaker': 'sys',
   'strategy': 'Others'},
  {'text': 'hi i am okay, a little bit sad though', 'speaker': 'usr'},
  {'text': 'Okay. I am very sorry to hear that! Do you want to tell me more about that?',
   'speaker': 'sys',
   'strategy': 'Question'},
  {'text': 'Well with the holidays coming up i have been very stressed and nervous about what i am going to do',
   'speaker': 'usr'},
  {'text': 'I really feel you there. Holidays are so hard.. especially with the way this year has been. Anything specific?',
   'speaker': 'sys',
   'strategy': 'Affirmation and R

In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/csgrad/sunilruf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import torch
def generate_content(tokenizer, messages):
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    tokenizer.pad_token = tokenizer.eos_token
    with torch.no_grad():
        outputs = model.generate(
                input_ids=input_ids,
                pad_token_id=tokenizer.eos_token_id,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
        )
        
        val = (tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("[/INST]")[-1])
    
    return val, input_ids

In [22]:
import torch
from tqdm import tqdm

max_length = 1024
stride = 128

#encodings = tokenizer.encode_plus(data['dialog'][1]['text'], return_tensors="pt").to('cuda')
tokenizer.pad_token = tokenizer.eos_token
seq_len = 1024

nlls = []
nlls_boh = []
prev_end_loc = 0
messages = []
i = 1
perplexity = []
for j in range(1):
    num = random.choice(range(3000, len(extes_data)))
    #num = 10000
    data = extes_data[num]['content']
    
    #data = data
    messages = []
    messages_boh = []
    for i in range(0, len(data), 2):
        for begin_loc in tqdm(range(0, seq_len, stride)):
            print(data[i])
            messages.append({"role": "user", "content":  data[i]['User']})
            messages_boh.append({"role": "user", "content":  remove_stopwords(data[i]['User'])})
            i+=2
            end_loc = min(begin_loc + max_length, seq_len)
            trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
            #input_ids = encodings.input_ids[:, begin_loc:end_loc].to('cuda')
            input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")
            input_ids_boh = tokenizer.apply_chat_template(messages_boh, tokenize=True, truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")
            target_ids = input_ids.clone()
            target_ids_boh = input_ids_boh.clone()
            target_ids[:, :-trg_len] = -100
            target_ids_boh[:, :-trg_len] = -100

            with torch.no_grad():
                
                
                val, _ = generate_content(tokenizer, messages)
                val_boh, _ = generate_content(tokenizer, messages_boh)
                outputs = model(input_ids, labels=target_ids)
                print(val)
                print(val_boh)
                messages.append({"role": "assistant", "content": val})
                print(outputs.logits.shape)
                outputs_boh = model(input_ids_boh, labels=target_ids_boh)
                print(outputs_boh.logits.shape)
                messages_boh.append({"role": "assistant", "content": remove_stopwords(val_boh)})
                    
                
                neg_log_likelihood = outputs.loss
                neg_log_likelihood_boh = outputs_boh.loss
                #print(neg_log_likelihood.shape)

            nlls.append(neg_log_likelihood)
            nlls_boh.append(neg_log_likelihood_boh)
            #print(len(nlls))
            prev_end_loc = end_loc
            if end_loc == seq_len:
                break

        ppl = torch.exp(torch.stack(nlls).mean())
        ppl_boh = torch.exp(torch.stack(nlls_boh).mean())
        print(ppl)
        print(ppl_boh)
        perplexity.append(ppl)
        

  0%|          | 0/8 [00:00<?, ?it/s]

{'User': 'Hey there', 'AI Strategy': '', 'AI': 'Hi, how are you doing today?'}


  0%|          | 0/8 [00:01<?, ?it/s]

 Hello! How can I help you today?
 Hello there! How can I help you today?
torch.Size([1, 10, 32000])
torch.Size([1, 9, 32000])


tensor(22.5263, device='cuda:0')
tensor(29.3700, device='cuda:0')


...

torch.Size([1, 51, 32000])
tensor(9.7453, device='cuda:0')
tensor(53.9628, device='cuda:0')


  0%|          | 0/8 [00:00<?, ?it/s]

{'User': "I'll definitely give that a try. It's important to approach the conversation with a level head and really listen to what they have to say. I hope it helps us to reconnect and find a way forward. Thanks for the suggestion!", 'AI Strategy': 'Affirmation', 'AI': "You're welcome! Taking a calm and open-minded approach can certainly create a space for healthier communication. Just remember that rebuilding a relationship takes time and effort from both sides. I'm here for you if you need any further support or guidance. Wishing you the best of luck!"}
 You're absolutely right. Approaching the conversation with empathy and a willingness to listen can go a long way in improving communication. I hope you find some progress and are able to reconnect with your loved one. You're doing great!
you're taking proactive steps, which is great. remember, communication takes effort from both sides. hope reconnecting brings positive change. best of luck!
torch.Size([1, 178, 32000])


  0%|          | 0/8 [00:06<?, ?it/s]

torch.Size([1, 109, 32000])
tensor(6.6948, device='cuda:0')
tensor(64.3374, device='cuda:0')


In [ ]:
data['dialog'][i]['text']

In [26]:
import torch
from tqdm import tqdm

max_length = 1024
stride = 128

#encodings = tokenizer.encode_plus(data['dialog'][1]['text'], return_tensors="pt").to('cuda')
tokenizer.pad_token = tokenizer.eos_token
seq_len = 1024
data = json.loads(dataset['test'][2]['text'])
nlls = []
prev_end_loc = 0
messages = []
messages_boh = []
i = 1
while i<29:
    for begin_loc in (range(0, seq_len, stride)):
        #print(data['dialog'][i]['text'])
        messages.append({"role": "user", "content":  data['dialog'][i]['text']})
        messages_boh.append({"role": "user", "content":  remove_stopwords(data['dialog'][i]['text'])})
        i+=2
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        #input_ids = encodings.input_ids[:, begin_loc:end_loc].to('cuda')
        input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")
        input_ids_boh = tokenizer.apply_chat_template(messages_boh, tokenize=True, truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")
        target_ids = input_ids.clone()
        target_ids_boh = input_ids_boh.clone()
        target_ids[:, :-trg_len] = -100
        target_ids_boh[:, :-trg_len] = -100

        with torch.no_grad():
            
            val, _ = generate_content(tokenizer, messages)
            val_boh, _ = generate_content(tokenizer, messages_boh)
            outputs = model(input_ids, labels=target_ids)
            print(val)
            print(val_boh)
            messages.append({"role": "assistant", "content": val})
            print(outputs.logits.shape)
            outputs_boh = model(input_ids_boh, labels=target_ids_boh)
            print(outputs_boh.logits.shape)
            messages_boh.append({"role": "assistant", "content": remove_stopwords(val_boh)})
                
            
            neg_log_likelihood = outputs.loss
            neg_log_likelihood_boh = outputs_boh.loss
            #print(neg_log_likelihood.shape)

        nlls.append(neg_log_likelihood)
        nlls_boh.append(neg_log_likelihood_boh)
            
        print(len(nlls))
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).mean())
    #print(ppl)
    ppl_boh = torch.exp(torch.stack(nlls_boh).mean())
    print(ppl)
    print(ppl_boh)
    perplexity.append(ppl)
    #messages.append({"role": "assistant", "content": val})

  0%|          | 0/8 [00:00<?, ?it/s]

I'd love to chat with you! What's been on your mind?


  0%|          | 0/8 [00:09<?, ?it/s]

 I can understand how that must be challenging for you. It sounds like you're really juggling a lot of responsibilities right now.
 Hello! Mind refers to the mental processes, consciousness, thoughts, and perceptions that we have as human beings. It's the complex system of cognitive abilities that enable us to learn, remember, think, problem-solve, and communicate. Is there a specific aspect of the mind you'd like to discuss?
torch.Size([1, 25, 32000])
torch.Size([1, 20, 32000])
1


tensor(8.3466, device='cuda:0')
tensor(72.4942, device='cuda:0')


...

That's so cool. Thanks for your help
 You're welcome! I'm glad I could provide some guidance and support. Remember, you deserve to have meaningful relationships and connections in your life. Don't give up on yourself or your happiness. Good luck with the apps!
you're welcome! Remember, you're not alone in this. Take small steps, be open, and I'm here for you.
torch.Size([1, 889, 32000])


  0%|          | 0/8 [00:07<?, ?it/s]


torch.Size([1, 494, 32000])
11
tensor(4.6386, device='cuda:0')
tensor(50.7945, device='cuda:0')


  0%|          | 0/8 [00:00<?, ?it/s]


IndexError: list index out of range